In [3]:
# Standard library imports
import itertools
import json
import logging
import sys
import time
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

# Third-party imports
import pulp
import pandas as pd
from tqdm import tqdm # Make sure this is installed: pip install tqdm

# Configure logging
logging.basicConfig(
 level=logging.INFO,
 format='%(asctime)s - %(levelname)s - %(message)s',
 handlers=[
 logging.FileHandler('scheduler.log'),
 logging.StreamHandler(sys.stdout)
 ]
)

@dataclass
class ScenarioParams:
 """Parameters defining a scenario"""
 resource_levels: Dict[str, int]
 sample_counts: Dict[str, int]

@dataclass
class ScenarioResult:
 """Results from running a scenario"""
 params: ScenarioParams
 makespan: Optional[int]
 total_cost: float
 samples_per_year: int
 cost_per_sample: float
 resource_utilization: Dict[str, float]
 error: Optional[str] = None # Track any errors that occurred

class SchedulerError(Exception):
 """Custom exception for scheduler-specific errors"""
 pass

def validate_scenario(scenario: dict, templates: List[dict]) -> None:
 """Validate that a scenario has necessary resources for the templates."""
 required_resources = set()
 for template in templates:
 for task in template:
 required_resources.add(task["resource"])
 
 available_resources = set(scenario['resource_capacities'].keys())
 missing_resources = required_resources - available_resources
 
 if missing_resources:
 raise SchedulerError(
 f"Missing required resources: {missing_resources}. "
 "These resources are needed for tasks in the templates."
 )

def process_scenario(scenario: dict, templates: List[dict], resource_unit_costs: Dict[str, float], 
 scenario_num: int, total_scenarios: int) -> dict:
 """Process a single scenario with error handling."""
 try:
 # Validate scenario
 validate_scenario(scenario, templates)
 
 all_tasks = []
 scenario_values = list(scenario.values())[:-1]
 for template, count, prefix in zip(templates, scenario_values, ["MET_", "CER_", "COMP_", "POLY_"]):
 all_tasks.extend(replicate_wbs_optimized(template, count, prefix))
 
 makespan, starts = schedule_wbs_discrete_optimized(
 tasks=all_tasks,
 resource_capacities=scenario['resource_capacities']
 )
 
 if makespan is None:
 raise SchedulerError("Failed to find feasible schedule")
 
 total_cost, usage = compute_resource_usage_cost(
 all_tasks, starts, resource_unit_costs, makespan
 )
 
 return {
 'scenario': scenario,
 'makespan': makespan,
 'total_cost': total_cost,
 'usage': usage,
 'error': None
 }
 
 except SchedulerError as e:
 logging.warning(f"Scenario {scenario_num}/{total_scenarios} failed: {str(e)}")
 return {
 'scenario': scenario,
 'makespan': None,
 'total_cost': 0,
 'usage': {},
 'error': str(e)
 }
 except Exception as e:
 logging.error(f"Unexpected error in scenario {scenario_num}/{total_scenarios}: {str(e)}")
 return {
 'scenario': scenario,
 'makespan': None,
 'total_cost': 0,
 'usage': {},
 'error': f"Unexpected error: {str(e)}"
 }

def run_scenarios_optimized(scenarios: List[dict], templates: List[dict], 
 resource_unit_costs: Dict[str, float]) -> List[dict]:
 """Run all scenarios in parallel with progress tracking."""
 total_scenarios = len(scenarios)
 completed = 0
 results = []
 
 logging.info(f"Starting optimization of {total_scenarios} scenarios")
 start_time = time.time()
 
 with ThreadPoolExecutor() as executor:
 # Submit all scenarios
 future_to_scenario = {
 executor.submit(
 process_scenario, 
 scenario, 
 templates, 
 resource_unit_costs,
 i + 1,
 total_scenarios
 ): i for i, scenario in enumerate(scenarios)
 }
 
 # Process completed scenarios with progress bar
 with tqdm(total=total_scenarios, desc="Processing scenarios") as pbar:
 for future in as_completed(future_to_scenario):
 scenario_idx = future_to_scenario[future]
 try:
 result = future.result()
 results.append(result)
 completed += 1
 
 # Update progress every 50 scenarios
 if completed % 50 == 0:
 elapsed = time.time() - start_time
 rate = completed / elapsed
 eta = (total_scenarios - completed) / rate if rate > 0 else 0
 logging.info(
 f"Completed {completed}/{total_scenarios} scenarios "
 f"({completed/total_scenarios*100:.1f}%) "
 f"Rate: {rate:.1f} scenarios/sec "
 f"ETA: {eta/60:.1f} minutes"
 )
 
 except Exception as e:
 logging.error(f"Error processing scenario {scenario_idx}: {str(e)}")
 results.append({
 'scenario': scenarios[scenario_idx],
 'makespan': None,
 'total_cost': 0,
 'usage': {},
 'error': str(e)
 })
 
 pbar.update(1)
 
 # Final summary
 end_time = time.time()
 total_time = end_time - start_time
 successful = sum(1 for r in results if r['error'] is None)
 
 logging.info(
 f"\nCompleted {total_scenarios} scenarios in {total_time:.1f} seconds\n"
 f"Successful: {successful}/{total_scenarios} ({successful/total_scenarios*100:.1f}%)\n"
 f"Failed: {total_scenarios-successful}/{total_scenarios} "
 f"({(total_scenarios-successful)/total_scenarios*100:.1f}%)"
 )
 
 return results

def generate_scenarios(
 resource_ranges: Dict[str, Tuple[int, int]], # e.g., {"Admin": (1,3)}
 sample_ranges: Dict[str, Tuple[int, int]], # e.g., {"metals": (0,10)}
 step_sizes: Dict[str, int] = None # Optional step sizes
) -> List[ScenarioParams]:
 """Generate all scenario combinations within given ranges"""
 if step_sizes is None:
 step_sizes = {k: 1 for k in {**resource_ranges, **sample_ranges}.keys()}
 
 # Generate resource level combinations
 resource_values = [
 range(start, end + 1, step_sizes.get(resource, 1))
 for resource, (start, end) in resource_ranges.items()
 ]
 resource_combinations = list(itertools.product(*resource_values))
 
 # Generate sample count combinations
 sample_values = [
 range(start, end + 1, step_sizes.get(sample_type, 1))
 for sample_type, (start, end) in sample_ranges.items()
 ]
 sample_combinations = list(itertools.product(*sample_values))
 
 # Create all possible combinations
 scenarios = []
 for res_combo in resource_combinations:
 for sample_combo in sample_combinations:
 resource_dict = dict(zip(resource_ranges.keys(), res_combo))
 sample_dict = dict(zip(sample_ranges.keys(), sample_combo))
 scenarios.append(ScenarioParams(
 resource_levels=resource_dict,
 sample_counts=sample_dict
 ))
 
 return scenarios
############################
# Critical Path Calculation
############################
def compute_earliest_starts(tasks):
 """Compute the earliest possible start times for each task using the critical path method."""
 earliest_starts = {}
 task_dict = {t["id"]: t for t in tasks}

 def get_earliest_start(task_id, memo=None):
 if memo is None:
 memo = {}
 if task_id in memo:
 return memo[task_id]
 
 task = task_dict[task_id]
 if not task["dependencies"]: # No dependencies
 memo[task_id] = 0
 return 0
 
 max_pred_finish = 0
 for pred_id in task["dependencies"]:
 if pred_id not in task_dict:
 raise ValueError(f"Task {task_id} references missing dependency {pred_id}")
 pred_start = get_earliest_start(pred_id, memo)
 pred_finish = pred_start + task_dict[pred_id]["duration"]
 max_pred_finish = max(max_pred_finish, pred_finish)
 
 memo[task_id] = max_pred_finish
 return max_pred_finish

 for task in tasks:
 earliest_starts[task["id"]] = get_earliest_start(task["id"])
 
 return earliest_starts

############################
# Optimized Discrete Scheduler
############################
def schedule_wbs_discrete_optimized(tasks, resource_capacities, time_horizon=None):
 """Schedule tasks with dependencies and resource constraints in discrete time using ILP."""
 # Compute earliest start times and critical path
 earliest_starts = compute_earliest_starts(tasks)
 
 # Compute tighter time horizon
 if time_horizon is None:
 time_horizon = max(
 earliest_starts[t["id"]] + t["duration"] for t in tasks
 )
 
 # Quick lookups
 task_ids = [t["id"] for t in tasks]
 durations = {t["id"]: t["duration"] for t in tasks}
 dependencies = {t["id"]: t["dependencies"] for t in tasks}
 resources = {t["id"]: t["resource"] for t in tasks}
 
 # Create the ILP model
 model = pulp.LpProblem("Discrete_WBS_Scheduling", pulp.LpMinimize)
 
 # Decision variables: Start times
 x = {}
 for i in task_ids:
 for t in range(earliest_starts[i], time_horizon - durations[i] + 1):
 x[i, t] = pulp.LpVariable(f"start_{i}_{t}", cat=pulp.LpBinary)
 
 # Makespan variable
 T = pulp.LpVariable("Makespan", lowBound=0, upBound=time_horizon, cat=pulp.LpInteger)
 
 # Constraints
 # 1. Each task must start exactly once
 for i in task_ids:
 model += pulp.lpSum(x[i, t] for t in range(earliest_starts[i], time_horizon - durations[i] + 1)) == 1
 
 # 2. Precedence constraints
 for i in task_ids:
 for j in dependencies[i]:
 dur_j = durations[j]
 for t_j in range(earliest_starts[j], time_horizon - dur_j + 1):
 finish_j = t_j + dur_j
 for t_i in range(finish_j, time_horizon - durations[i] + 1):
 if (j, t_j) in x and (i, t_i) in x:
 model += x[j, t_j] + x[i, t_i] <= 1
 
 # 3. Resource constraints
 resource_tasks = defaultdict(list)
 for t in tasks:
 resource_tasks[t["resource"]].append(t["id"])
 
 for r, cap in resource_capacities.items():
 tasks_r = resource_tasks[r]
 if not tasks_r:
 continue
 for u in range(time_horizon):
 active_sum = []
 for i in tasks_r:
 dur_i = durations[i]
 for t_val in range(earliest_starts[i], time_horizon - dur_i + 1):
 if t_val <= u < t_val + dur_i and (i, t_val) in x:
 active_sum.append(x[i, t_val])
 if active_sum:
 model += pulp.lpSum(active_sum) <= cap
 
 # 4. Makespan constraints
 for i in task_ids:
 for t_val in range(earliest_starts[i], time_horizon - durations[i] + 1):
 if (i, t_val) in x:
 model += T >= t_val + durations[i] * x[i, t_val]
 
 # Objective: Minimize makespan
 model += T, "Minimize_Makespan"
 
 # Solve
 solver = pulp.PULP_CBC_CMD(msg=0, timeLimit=300)
 model.solve(solver)
 
 # Extract solution
 start_times = {}
 for i in task_ids:
 for t_val in range(earliest_starts[i], time_horizon - durations[i] + 1):
 if (i, t_val) in x and pulp.value(x[i, t_val]) > 0.5:
 start_times[i] = t_val
 break
 
 makespan_value = int(round(pulp.value(T))) if pulp.value(T) is not None else None
 return makespan_value, start_times

############################
# Replicate WBS
############################
def replicate_wbs_optimized(template, count, prefix):
 """Replicate a WBS template multiple times."""
 if count == 0:
 return []
 replicated_tasks = []
 for n in range(count):
 instance_suffix = f"_{n}"
 for task in template:
 replicated_tasks.append({
 "id": f"{prefix}{task['id']}{instance_suffix}",
 "duration": task["duration"],
 "dependencies": [f"{prefix}{dep}{instance_suffix}" for dep in task["dependencies"]],
 "resource": task["resource"]
 })
 return replicated_tasks

############################
# Resource Usage and Cost
############################
def compute_resource_usage_cost(tasks, start_times, resource_unit_costs, makespan):
 """Calculate resource usage and cost over the makespan."""
 resource_usage = defaultdict(lambda: defaultdict(int))
 for task_id, start_time in start_times.items():
 task = next(t for t in tasks if t["id"] == task_id)
 for t in range(start_time, start_time + task["duration"]):
 if t <= makespan:
 resource_usage[task["resource"]][t] += 1
 
 total_cost = sum(
 usage * resource_unit_costs.get(r, 0.0)
 for r, times in resource_usage.items()
 for usage in times.values()
 )
 return total_cost, dict(resource_usage)

############################
# Run Scenarios
############################
def process_scenario(scenario, templates, resource_unit_costs):
 """Process a single scenario."""
 all_tasks = []
 # Convert dict_values to list before slicing
 scenario_values = list(scenario.values())[:-1]
 for template, count, prefix in zip(templates, scenario_values, ["MET_", "CER_", "COMP_", "POLY_"]):
 all_tasks.extend(replicate_wbs_optimized(template, count, prefix))
 
 makespan, starts = schedule_wbs_discrete_optimized(
 tasks=all_tasks,
 resource_capacities=scenario['resource_capacities']
 )
 total_cost, usage = compute_resource_usage_cost(all_tasks, starts, resource_unit_costs, makespan)
 return {'scenario': scenario, 'makespan': makespan, 'total_cost': total_cost, 'usage': usage}

def process_and_save_results(scenarios: List[ScenarioParams], 
 raw_results: List[dict], 
 output_file: str) -> pd.DataFrame:
 """
 Process raw scheduling results into final format and save to CSV.
 
 Args:
 scenarios: List of original ScenarioParams objects
 raw_results: List of raw results from scheduler
 output_file: Path to save CSV output
 
 Returns:
 DataFrame containing processed results
 """
 final_results = []
 
 for scenario, result in zip(scenarios, raw_results):
 if result['error'] is not None:
 # Handle failed scenarios
 final_results.append(ScenarioResult(
 params=scenario,
 makespan=None,
 total_cost=0.0,
 samples_per_year=0,
 cost_per_sample=0.0,
 resource_utilization={},
 error=result['error']
 ))
 continue
 
 # Calculate annual throughput (assuming makespan is in days)
 total_samples = sum(scenario.sample_counts.values())
 if result['makespan'] > 0:
 samples_per_year = int(365 * total_samples / result['makespan'])
 else:
 samples_per_year = 0
 
 # Calculate cost per sample
 if total_samples > 0:
 cost_per_sample = result['total_cost'] / total_samples
 else:
 cost_per_sample = 0.0
 
 # Calculate resource utilization
 utilization = {}
 for resource, usage_dict in result['usage'].items():
 max_possible_usage = scenario.resource_levels[resource] * result['makespan']
 total_usage = sum(usage_dict.values())
 if max_possible_usage > 0:
 utilization[resource] = total_usage / max_possible_usage
 else:
 utilization[resource] = 0.0
 
 final_results.append(ScenarioResult(
 params=scenario,
 makespan=result['makespan'],
 total_cost=result['total_cost'],
 samples_per_year=samples_per_year,
 cost_per_sample=cost_per_sample,
 resource_utilization=utilization,
 error=None
 ))
 
 # Convert to DataFrame for saving
 records = []
 for result in final_results:
 record = {
 # Resource levels
 **{f"resource_{k}": v for k, v in result.params.resource_levels.items()},
 # Sample counts
 **{f"samples_{k}": v for k, v in result.params.sample_counts.items()},
 # Results
 "makespan": result.makespan,
 "total_cost": result.total_cost,
 "samples_per_year": result.samples_per_year,
 "cost_per_sample": result.cost_per_sample,
 # Resource utilization
 **{f"utilization_{k}": v for k, v in result.resource_utilization.items()},
 # Error tracking
 "error": result.error
 }
 records.append(record)
 
 # Create DataFrame and save
 df = pd.DataFrame(records)
 df.to_csv(output_file, index=False)
 
 # Log summary statistics
 successful = df['error'].isna().sum()
 total = len(df)
 logging.info(f"\nResults Summary:")
 logging.info(f"Total scenarios processed: {total}")
 logging.info(f"Successful scenarios: {successful} ({successful/total*100:.1f}%)")
 logging.info(f"Failed scenarios: {total-successful} ({(total-successful)/total*100:.1f}%)")
 if successful > 0:
 logging.info(f"Average samples per year: {df['samples_per_year'].mean():.1f}")
 logging.info(f"Average cost per sample: ${df['cost_per_sample'].mean():.2f}")
 logging.info(f"Results saved to: {output_file}")
 
 return df

def run_capacity_analysis(
 templates: List[dict],
 resource_ranges: Dict[str, Tuple[int, int]],
 sample_ranges: Dict[str, Tuple[int, int]],
 resource_unit_costs: Dict[str, float],
 step_sizes: Dict[str, int] = None,
 output_file: str = "capacity_results.csv"
) -> pd.DataFrame:
 """Run complete capacity analysis across all scenarios."""
 # Generate scenarios
 scenarios = generate_scenarios(resource_ranges, sample_ranges, step_sizes)
 logging.info(f"Generated {len(scenarios)} scenarios to evaluate")
 
 # Convert scenarios to scheduler format
 scheduler_scenarios = []
 for scenario in scenarios:
 scheduler_scenario = {
 "metals_count": scenario.sample_counts.get("metals", 0),
 "ceramics_count": scenario.sample_counts.get("ceramics", 0),
 "composites_count": scenario.sample_counts.get("composites", 0),
 "polymer_count": scenario.sample_counts.get("polymers", 0),
 "resource_capacities": scenario.resource_levels
 }
 scheduler_scenarios.append(scheduler_scenario)
 
 # Run scenarios
 results = run_scenarios_optimized(scheduler_scenarios, templates, resource_unit_costs)
 
 # Convert results to ScenarioResult format and save
 final_results = process_and_save_results(scenarios, results, output_file)
 
 return final_results

def run_scenarios_optimized(scenarios, templates, resource_unit_costs):
 """Run all scenarios in parallel."""
 with ThreadPoolExecutor() as executor:
 results = executor.map(lambda s: process_scenario(s, templates, resource_unit_costs), scenarios)
 return list(results)



def save_results(results: List[ScenarioResult], filename: str):
 """Save scenario results to CSV"""
 records = []
 for result in results:
 record = {
 # Resource levels
 **{f"resource_{k}": v for k, v in result.params.resource_levels.items()},
 # Sample counts
 **{f"samples_{k}": v for k, v in result.params.sample_counts.items()},
 # Results
 "makespan": result.makespan,
 "total_cost": result.total_cost,
 "samples_per_year": result.samples_per_year,
 "cost_per_sample": result.cost_per_sample,
 # Resource utilization
 **{f"utilization_{k}": v for k, v in result.resource_utilization.items()}
 }
 records.append(record)
 
 df = pd.DataFrame(records)
 df.to_csv(filename, index=False)
 return df

# Example usage:
if __name__ == "__main__":
 # Define ranges to test
 resource_ranges = {
 "Admin": (1, 3),
 "LabTech": (1, 2),
 "MaterialsScientist": (1, 2)
 }
 
 sample_ranges = {
 "metals": (0, 10),
 "ceramics": (0, 10),
 "composites": (0, 8),
 "polymers": (0, 8)
 }
 
 # Optional: Define step sizes to reduce combinations
 step_sizes = {
 "metals": 2,
 "ceramics": 2,
 "composites": 2,
 "polymers": 2
 }
 
 # Generate scenarios
 scenarios = generate_scenarios(resource_ranges, sample_ranges, step_sizes)
 print(f"Generated {len(scenarios)} scenarios")
 
 # These would be your actual results after running the scenarios
 example_result = ScenarioResult(
 params=scenarios[0],
 makespan=100,
 total_cost=50000.0,
 samples_per_year=45,
 cost_per_sample=1111.11,
 resource_utilization={"Admin": 0.85, "LabTech": 0.92, "MaterialsScientist": 0.75}
 )
 
 # Save results
 save_results([example_result], "scenario_results.csv")
 
import pulp
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass
from typing import Dict, List, Tuple
import pandas as pd
import time

# [Previous scenario generation code remains the same through the classes]
@dataclass
class ScenarioParams:
 resource_levels: Dict[str, int]
 sample_counts: Dict[str, int]

@dataclass
class ScenarioResult:
 params: ScenarioParams
 makespan: int
 total_cost: float
 samples_per_year: int
 cost_per_sample: float
 resource_utilization: Dict[str, float]

def run_capacity_analysis(
 templates: List[dict],
 resource_ranges: Dict[str, Tuple[int, int]],
 sample_ranges: Dict[str, Tuple[int, int]],
 resource_unit_costs: Dict[str, float],
 step_sizes: Dict[str, int] = None, # Optional - set to None to use step size of 1
 output_file: str = "capacity_results.csv"
) -> pd.DataFrame:
 """
 Run complete capacity analysis across all scenarios.
 
 Args:
 templates: List of WBS templates [metals, ceramics, composites, polymers]
 resource_ranges: Dict of resource min/max e.g., {"Admin": (1,3)}
 sample_ranges: Dict of sample type min/max e.g., {"metals": (0,10)}
 resource_unit_costs: Dict of costs per resource unit
 step_sizes: Dict of step sizes for each parameter (optional)
 output_file: Where to save results
 """
 # Generate scenarios
 scenarios = generate_scenarios(resource_ranges, sample_ranges, step_sizes)
 print(f"Generated {len(scenarios)} scenarios to evaluate")
 
 # Convert scenarios to format needed by scheduler
 scheduler_scenarios = []
 for scenario in scenarios:
 scheduler_scenario = {
 "metals_count": scenario.sample_counts.get("metals", 0),
 "ceramics_count": scenario.sample_counts.get("ceramics", 0),
 "composites_count": scenario.sample_counts.get("composites", 0),
 "polymer_count": scenario.sample_counts.get("polymers", 0),
 "resource_capacities": scenario.resource_levels
 }
 scheduler_scenarios.append(scheduler_scenario)
 
 # Run scenarios
 start_time = time.time()
 results = run_scenarios_optimized(scheduler_scenarios, templates, resource_unit_costs)
 end_time = time.time()
 print(f"Completed {len(scenarios)} scenarios in {end_time - start_time:.1f} seconds")
 
 # Convert results to ScenarioResult format
 final_results = []
 for scenario, result in zip(scenarios, results):
 if result['makespan'] is None: # Handle infeasible scenarios
 continue
 
 # Calculate samples per year (assuming makespan is in days)
 total_samples = sum(scenario.sample_counts.values())
 samples_per_year = int(365 * total_samples / result['makespan'])
 
 # Calculate cost per sample
 cost_per_sample = result['total_cost'] / total_samples if total_samples > 0 else 0
 
 # Calculate resource utilization
 utilization = {}
 for resource, usage_dict in result['usage'].items():
 max_possible_usage = scenario.resource_levels[resource] * result['makespan']
 total_usage = sum(usage_dict.values())
 utilization[resource] = total_usage / max_possible_usage if max_possible_usage > 0 else 0
 
 final_results.append(ScenarioResult(
 params=scenario,
 makespan=result['makespan'],
 total_cost=result['total_cost'],
 samples_per_year=samples_per_year,
 cost_per_sample=cost_per_sample,
 resource_utilization=utilization
 ))
 
 # Save results
 df = save_results(final_results, output_file)
 return df

#
# ============= YOUR WBS TEMPLATES =============
#
metals_template = [
 # ------ Sample Management ------
 {"id": "1.1", "name": "Receive and Log Samples", "duration": 4, "dependencies": [], "resource": "Administrative Support Specialist"},
 {"id": "1.2", "name": "Assign Unique Identifiers", "duration": 2, "dependencies": ["1.1"], "resource": "Administrative Support Specialist"},
 {"id": "1.3", "name": "Photograph Initial Condition", "duration": 2, "dependencies": ["1.2"], "resource": "Laboratory Technician"},
 {"id": "1.4", "name": "Write Analysis Plan", "duration": 3, "dependencies": ["1.3"], "resource": "Materials Scientist (Metals)"},
 {"id": "1.5", "name": "Store Samples Appropriately", "duration": 1, "dependencies": ["1.4"], "resource": "Administrative Support Specialist"},
 {"id": "1.6", "name": "Document Chain of Custody", "duration": 1, "dependencies": ["1.5"], "resource": "Administrative Support Specialist"},
 # ------ Sample Preparation ------
 {"id": "2.1", "name": "Cut Subsamples for Mounting", "duration": 3, "dependencies": ["1.6"], "resource": "Laboratory Technician"},
 {"id": "2.2", "name": "Cut Subsamples for Light El.", "duration": 3, "dependencies": ["1.6"], "resource": "Laboratory Technician"},
 {"id": "2.3", "name": "Cut for ICP-OES/ICP-MS", "duration": 3, "dependencies": ["1.6"], "resource": "Laboratory Technician"},
 {"id": "2.4", "name": "Clean Cut Subsamples", "duration": 2, "dependencies": ["2.1","2.2","2.3"], "resource": "Laboratory Technician"},
 {"id": "2.5", "name": "Mount Subsample in Epoxy", "duration": 3, "dependencies": ["2.1"], "resource": "Laboratory Technician"},
 {"id": "2.6", "name": "Grind & Polish Mounted Subsmpl", "duration": 4, "dependencies": ["2.5"], "resource": "Materials Scientist (Metals)"},
 {"id": "2.7", "name": "Digest Sample with Acid/MW", "duration": 4, "dependencies": ["2.4"], "resource": "Analytical Instrument Specialist"},
 {"id": "2.8", "name": "Prep ICP Standards/Blanks", "duration": 2, "dependencies": [], "resource": "Analytical Instrument Specialist"},
 {"id": "2.9", "name": "Coat Sample for SEM/EBSD", "duration": 2, "dependencies": ["2.6"], "resource": "Laboratory Technician"},
 {"id": "2.10","name": "Label & Catalog Prepared Smpls", "duration": 2, "dependencies": ["2.3","2.6","2.7","2.9"], "resource": "Administrative Support Specialist"},
 # ------ Sample Analysis ------
 {"id": "3.1", "name": "Optical Microscope", "duration": 2, "dependencies": ["2.6"], "resource": "Materials Scientist (Metals)"},
 {"id": "3.2", "name": "SEM Imaging", "duration": 4, "dependencies": ["2.9"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.3", "name": "SEM-EDS Bulk Composition", "duration": 4, "dependencies": ["3.2"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.4", "name": "SEM-EBSD Grain Analysis", "duration": 4, "dependencies": ["3.2"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.5", "name": "Density via Pycnometer", "duration": 3, "dependencies": ["2.4"], "resource": "Laboratory Technician"},
 {"id": "3.6", "name": "Composition XRF", "duration": 3, "dependencies": ["2.4"], "resource": "Laboratory Technician"},
 {"id": "3.7", "name": "Composition SparkOES", "duration": 3, "dependencies": ["2.6"], "resource": "Laboratory Technician"},
 {"id": "3.8", "name": "Light Elements (CHNOS)", "duration": 3, "dependencies": ["2.4"], "resource": "Analytical Instrument Specialist"},
 {"id": "3.9", "name": "Hardness Microindentation", "duration": 3, "dependencies": ["2.6"], "resource": "Materials Scientist (Metals)"},
 {"id": "3.10","name": "ICP-MS/ICP-OES", "duration": 8, "dependencies": [], "resource": "Analytical Instrument Specialist"},
 # ------ Data Analysis ------
 {"id": "4.1", "name": "Compile Analytical Results", "duration": 2, "dependencies": ["3.1","3.2","3.3","3.4","3.5","3.6","3.7","3.8","3.9"], "resource": "Materials Scientist (Metals)"},
 {"id": "4.2", "name": "Cross-Validate Composition", "duration": 6, "dependencies": ["3.3","3.4","3.5","3.6","3.7","3.8","3.9"], "resource": "Materials Scientist (Metals)"},
 {"id": "4.3", "name": "DB Match or Similar Alloy", "duration": 4, "dependencies": [], "resource": "Materials Scientist (Metals)"},
 {"id": "4.4", "name": "Prepare Final Figures", "duration": 3, "dependencies": ["4.3"], "resource": "Materials Scientist (Metals)"},
 {"id": "4.5", "name": "Perform Peer Review", "duration": 2, "dependencies": ["4.4"], "resource": "Materials Scientist (Metals)"},
 {"id": "4.6", "name": "QA Review of Data/Results", "duration": 1, "dependencies": ["4.5"], "resource": "Quality Assurance Officer"},
 {"id": "4.7", "name": "Draft Report to Client", "duration": 2, "dependencies": ["4.6"], "resource": "Project Manager"},
 {"id": "4.8", "name": "Update Report Per Feedback", "duration": 2, "dependencies": ["4.7"], "resource": "Project Manager"},
 {"id": "4.9", "name": "Enter Results into Database", "duration": 2, "dependencies": ["4.8"], "resource": "Administrative Support Specialist"},
]

ceramics_template = [
 # ------ Sample Management ------
 {"id": "1.1", "name": "Receive and Log Samples", "duration": 4, "dependencies": [], "resource": "Administrative Support Specialist"},
 {"id": "1.2", "name": "Assign Unique Identifiers", "duration": 2, "dependencies": ["1.1"], "resource": "Administrative Support Specialist"},
 {"id": "1.3", "name": "Photograph Initial Condition", "duration": 2, "dependencies": ["1.2"], "resource": "Laboratory Technician"},
 {"id": "1.4", "name": "Write Analysis Plan", "duration": 3, "dependencies": ["1.3"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "1.5", "name": "Store Samples Appropriately", "duration": 1, "dependencies": ["1.4"], "resource": "Administrative Support Specialist"},
 {"id": "1.6", "name": "Document Chain of Custody", "duration": 1, "dependencies": ["1.5"], "resource": "Administrative Support Specialist"},
 # ------ Sample Preparation ------
 {"id": "2.1", "name": "Cut Subsamples for Mounting", "duration": 3, "dependencies": ["1.6"], "resource": "Laboratory Technician"},
 {"id": "2.2", "name": "Cut Subsamples for ICP", "duration": 3, "dependencies": ["1.6"], "resource": "Laboratory Technician"},
 {"id": "2.3", "name": "Clean Cut Subsamples", "duration": 2, "dependencies": ["2.1","2.2"], "resource": "Laboratory Technician"},
 {"id": "2.4", "name": "Mount Subsample in Epoxy", "duration": 3, "dependencies": ["2.3"], "resource": "Laboratory Technician"},
 {"id": "2.5", "name": "Crush & Sieve for Powder", "duration": 4, "dependencies": ["2.3"], "resource": "Laboratory Technician"},
 {"id": "2.6", "name": "Grind & Polish Mounted Subsmpl", "duration": 4, "dependencies": ["2.4"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "2.7", "name": "Digest Sample (Acid/MW)", "duration": 4, "dependencies": ["2.3"], "resource": "Analytical Instrument Specialist"},
 {"id": "2.8", "name": "Prep ICP Standards/Blanks", "duration": 2, "dependencies": [], "resource": "Analytical Instrument Specialist"},
 {"id": "2.9", "name": "Coat Sample for SEM/EBSD", "duration": 2, "dependencies": ["2.6"], "resource": "Laboratory Technician"},
 {"id": "2.10","name": "Label & Catalog Prepared Smpls", "duration": 2, "dependencies": ["2.5","2.3","2.6","2.7","2.9"], "resource": "Administrative Support Specialist"},
 # ------ Sample Analysis ------
 {"id": "3.1", "name": "FTIR for Composition", "duration": 4, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.2", "name": "Raman for Crystalline Phases", "duration": 4, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.3", "name": "SEM Imaging (Microstructure)", "duration": 6, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.4", "name": "XRD for Phase Info", "duration": 5, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.5", "name": "TGA for Thermal Stability", "duration": 5, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.6", "name": "DSC for Thermal Transitions", "duration": 5, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.7", "name": "Microindentation (Knoop)", "duration": 6, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "3.8", "name": "SEM-EDS for Elemental Comp", "duration": 6, "dependencies": ["2.10"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.9", "name": "ICP-OES/ICP-MS for Trace Elem", "duration": 6, "dependencies": ["2.10"], "resource": "Materials Scientist (Ceramics)"},
 # ------ Data Analysis ------
 {"id": "4.1", "name": "Compile Analytical Results", "duration": 2, "dependencies": ["3.3","3.4","3.5","3.6","3.7","3.8","3.9"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "4.2", "name": "Interpret Data (Ceramic Props)", "duration": 6, "dependencies": ["4.1"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "4.3", "name": "Cross-Validate Across Techniques", "duration": 6, "dependencies": ["4.2"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "4.4", "name": "Prepare Final Figures", "duration": 3, "dependencies": ["4.3"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "4.5", "name": "Perform Peer Review", "duration": 2, "dependencies": ["4.4"], "resource": "Materials Scientist (Ceramics)"},
 {"id": "4.6", "name": "QA Review of Data/Results", "duration": 2, "dependencies": ["4.5"], "resource": "Quality Assurance Officer"},
 {"id": "4.7", "name": "Draft Report to Client", "duration": 2, "dependencies": ["4.6"], "resource": "Project Manager"},
 {"id": "4.8", "name": "Update Report Per Feedback", "duration": 2, "dependencies": ["4.7"], "resource": "Project Manager"},
 {"id": "4.9", "name": "Enter Results into Database", "duration": 2, "dependencies": ["4.8"], "resource": "Administrative Support Specialist"},
]

composites_template = [
 # ------ Sample Management ------
 {"id": "1.1", "name": "Receive and Log Samples", "duration": 4, "dependencies": [], "resource": "Administrative Support Specialist"},
 {"id": "1.2", "name": "Assign Unique Identifiers", "duration": 2, "dependencies": ["1.1"], "resource": "Administrative Support Specialist"},
 {"id": "1.3", "name": "Photograph Initial Condition", "duration": 2, "dependencies": ["1.2"], "resource": "Laboratory Technician"},
 {"id": "1.4", "name": "Write Analysis Plan", "duration": 3, "dependencies": ["1.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "1.5", "name": "Store Samples Appropriately", "duration": 1, "dependencies": ["1.4"], "resource": "Administrative Support Specialist"},
 {"id": "1.6", "name": "Document Chain of Custody", "duration": 1, "dependencies": ["1.5"], "resource": "Administrative Support Specialist"},
 # ------ Sample Preparation ------
 {"id": "2.1", "name": "Section and Polish", "duration": 3, "dependencies": ["1.6"], "resource": "Advanced Imaging Specialist"},
 {"id": "2.2", "name": "Embed Samples in Resin", "duration": 3, "dependencies": ["1.6"], "resource": "Laboratory Technician"},
 {"id": "2.3", "name": "Label & Catalog Prepared Smpls", "duration": 3, "dependencies": ["1.6"], "resource": "Administrative Support Specialist"},
 # ------ Sample Analysis ------
 {"id": "3.1", "name": "FTIR for Matrix", "duration": 4, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.2", "name": "Raman for Fibers", "duration": 4, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.3", "name": "Raman for Matrix", "duration": 4, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.4.1","name": "SEM-EDS Fiber Comp", "duration": 6, "dependencies": ["2.3"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.4.2","name": "SEM-EDS Matrix Comp", "duration": 6, "dependencies": ["2.3"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.5", "name": "TGA (Thermal Stability/Filler)", "duration": 5, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.6", "name": "DSC (Polymer Transition)", "duration": 5, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.7", "name": "PYGCMS (Matrix Composition)", "duration": 6, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.8", "name": "ICP-OES/ICP-MS (Elements)", "duration": 8, "dependencies": ["2.3"], "resource": "Analytical Instrument Specialist"},
 {"id": "3.9", "name": "Fiber Analysis (Optical/SEM)", "duration": 8, "dependencies": ["2.3"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.10","name": "Measure Density (Pycnometer)", "duration": 3, "dependencies": ["2.3"], "resource": "Laboratory Technician"},
 # ------ Data Analysis ------
 {"id": "4.1", "name": "Cross-Validate Composition", "duration": 6, "dependencies": ["3.3","3.4","3.5","3.6","3.7","3.8","3.9"], "resource": "Materials Scientist (Composites)"},
 {"id": "4.2", "name": "Prepare Final Figures", "duration": 3, "dependencies": ["4.1"], "resource": "Materials Scientist (Composites)"},
 {"id": "4.3", "name": "Perform Peer Review", "duration": 2, "dependencies": ["4.2"], "resource": "Materials Scientist (Composites)"},
 {"id": "4.4", "name": "QA Review of Data/Results", "duration": 1, "dependencies": ["4.3"], "resource": "Quality Assurance Officer"},
 {"id": "4.5", "name": "Draft Report to Client", "duration": 2, "dependencies": ["4.4"], "resource": "Project Manager"},
 {"id": "4.6", "name": "Update Report Per Feedback", "duration": 2, "dependencies": ["4.5"], "resource": "Project Manager"},
 {"id": "4.7", "name": "Enter Results into Database", "duration": 2, "dependencies": ["4.6"], "resource": "Administrative Support Specialist"},
]

polymers_template = [
 # ------ Sample Management ------
 {"id": "1.1", "name": "Receive and Log Samples", "duration": 4, "dependencies": [], "resource": "Administrative Support Specialist"},
 {"id": "1.2", "name": "Assign Unique Identifiers", "duration": 2, "dependencies": ["1.1"], "resource": "Administrative Support Specialist"},
 {"id": "1.3", "name": "Photograph Initial Condition", "duration": 2, "dependencies": ["1.2"], "resource": "Laboratory Technician"},
 {"id": "1.4", "name": "Write Analysis Plan", "duration": 3, "dependencies": ["1.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "1.5", "name": "Store Samples Appropriately", "duration": 1, "dependencies": ["1.4"], "resource": "Administrative Support Specialist"},
 {"id": "1.6", "name": "Document Chain of Custody", "duration": 1, "dependencies": ["1.5"], "resource": "Administrative Support Specialist"},
 # ------ Sample Preparation ------
 {"id": "2.1", "name": "Cut Subsamples for Mounting", "duration": 3, "dependencies": [], "resource": "Laboratory Technician"},
 {"id": "2.2", "name": "Cut Subsamples for ICP", "duration": 3, "dependencies": [], "resource": "Laboratory Technician"},
 {"id": "2.3", "name": "Clean Cut Subsamples", "duration": 2, "dependencies": ["2.1","2.2"], "resource": "Laboratory Technician"},
 {"id": "2.4", "name": "Mount Subsample in Epoxy", "duration": 3, "dependencies": ["2.1"], "resource": "Laboratory Technician"},
 {"id": "2.5", "name": "Grind & Polish Mounted Subsmpl", "duration": 4, "dependencies": ["2.4"], "resource": "Materials Scientist (Composites)"},
 {"id": "2.6", "name": "Digest Sample (Acid/MW)", "duration": 4, "dependencies": ["2.3"], "resource": "Analytical Instrument Specialist"},
 {"id": "2.7", "name": "Prep ICP Standards/Blanks", "duration": 2, "dependencies": [], "resource": "Analytical Instrument Specialist"},
 {"id": "2.8", "name": "Coat Sample for SEM/EBSD", "duration": 2, "dependencies": ["2.5"], "resource": "Laboratory Technician"},
 {"id": "2.9", "name": "Label & Catalog Prepared Smpls", "duration": 2, "dependencies": ["2.2","2.5","2.6","2.8"], "resource": "Administrative Support Specialist"},
 # ------ Sample Analysis ------
 {"id": "3.1", "name": "Optical Microscope", "duration": 2, "dependencies": ["2.5"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.2", "name": "SEM Imaging", "duration": 4, "dependencies": ["2.8"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.3", "name": "SEM-EDS (Polymer Comp)", "duration": 4, "dependencies": ["3.2"], "resource": "Advanced Imaging Specialist"},
 {"id": "3.4", "name": "FTIR for Composition", "duration": 4, "dependencies": ["2.5"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.5", "name": "Raman for Composition", "duration": 4, "dependencies": ["2.5"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.6", "name": "Measure Density (Pycnometer)", "duration": 3, "dependencies": ["2.3"], "resource": "Laboratory Technician"},
 {"id": "3.7", "name": "Hardness Microindentation", "duration": 3, "dependencies": ["2.5"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.8", "name": "TGA (Thermal Stability/Filler)", "duration": 5, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.9", "name": "DSC (Polymer Transition)", "duration": 5, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.10","name": "PYGCMS (Matrix)", "duration": 6, "dependencies": ["2.3"], "resource": "Materials Scientist (Composites)"},
 {"id": "3.11","name": "ICP-MS/ICP-OES", "duration": 8, "dependencies": ["2.6","2.7"], "resource": "Analytical Instrument Specialist"},
 # ------ Data Analysis ------
 {"id": "4.1", "name": "Cross-Validate Composition", "duration": 6, "dependencies": ["3.3","3.4","3.5","3.6","3.7","3.8","3.9"], "resource": "Materials Scientist (Composites)"},
 {"id": "4.2", "name": "Prepare Final Figures", "duration": 3, "dependencies": ["4.1"], "resource": "Materials Scientist (Composites)"},
 {"id": "4.3", "name": "Perform Peer Review", "duration": 2, "dependencies": ["4.2"], "resource": "Materials Scientist (Composites)"},
 {"id": "4.4", "name": "QA Review of Data/Results", "duration": 1, "dependencies": ["4.3"], "resource": "Quality Assurance Officer"},
 {"id": "4.5", "name": "Draft Report to Client", "duration": 2, "dependencies": ["4.4"], "resource": "Project Manager"},
 {"id": "4.6", "name": "Update Report Per Feedback", "duration": 2, "dependencies": ["4.5"], "resource": "Project Manager"},
 {"id": "4.7", "name": "Enter Results into Database", "duration": 2, "dependencies": ["4.6"], "resource": "Administrative Support Specialist"},
]

if __name__ == "__main__":
 logging.info("Starting capacity analysis")
 
 templates = [metals_template, ceramics_template, composites_template, polymers_template]
 
 resource_ranges = {
 "Administrative Support Specialist": (1, 4),
 "Laboratory Technician": (1, 3),
 "Materials Scientist (Metals)": (1, 2),
 "Materials Scientist (Ceramics)": (1, 2),
 "Materials Scientist (Composites)": (1, 2),
 "Advanced Imaging Specialist": (1, 2),
 "Analytical Instrument Specialist": (1, 2),
 "Quality Assurance Officer": (1, 1),
 "Project Manager": (1, 1)
 }
 
 sample_ranges = {
 "metals": (0, 20),
 "ceramics": (0, 20),
 "composites": (0, 15),
 "polymers": (0, 15)
 }
 
 step_sizes = {
 "metals": 4,
 "ceramics": 4,
 "composites": 3,
 "polymers": 3,
 **{r: 1 for r in resource_ranges.keys()}
 }
 
 resource_unit_costs = {
 "Administrative Support Specialist": 75,
 "Laboratory Technician": 100,
 "Materials Scientist (Metals)": 150,
 "Materials Scientist (Ceramics)": 150,
 "Materials Scientist (Composites)": 150,
 "Advanced Imaging Specialist": 150,
 "Analytical Instrument Specialist": 125,
 "Quality Assurance Officer": 125,
 "Project Manager": 175
 }
 
 try:
 results = run_capacity_analysis(
 templates=templates,
 resource_ranges=resource_ranges,
 sample_ranges=sample_ranges,
 resource_unit_costs=resource_unit_costs,
 step_sizes=step_sizes,
 output_file="capacity_results.csv"
 )
 logging.info("Analysis completed successfully")
 
 except Exception as e:
 logging.error(f"Analysis failed: {str(e)}")

IndentationError: expected an indented block after 'for' statement on line 51 (1499599134.py, line 52)